In [1]:
import copy
import time
import math

# parameters
path       = "Skin_NonSkin.txt"  # file path
splitChar  = "\t"                # split the line

k          = 2                   # k clusters
threshold  = 1e-4                # converge or not
dimension  = 3                   # dimensions of data points

# variables
centers    = []                  # centers of clusters 
costTotal  = 9999999999          # total cost

roundCount = 0                   # how many times (iteration)

In [2]:
# create random data
# dimension = 3
# k = 3

import random

def createData(dataCnt):

    dataList = []

    for i in range(dataCnt-dataCnt/10):
        s = random.randint(1,10)
        temp = []
        if s<6:
            temp.append(random.randint(10,50))
            temp.append(random.randint(10,60))
            temp.append(random.randint(20,30))
            temp.append(1)
        else:
            temp.append(random.randint(10,110))
            temp.append(random.randint(70,80))
            temp.append(random.randint(120,130))
            temp.append(2)
        dataList.append(temp)

    # add something
    for i in range(dataCnt/10):
        temp = []
        temp.append(random.randint(30,40))
        temp.append(random.randint(10,30))
        temp.append(random.randint(220,230))
        temp.append(3)
        dataList.append(temp)
        
    return dataList

In [3]:
# functions

# 2018/03/09
# a list : str->int
def intList(l):
    lNew = []
    for s in l:
        lNew.append(int(s))
    return lNew

# 2018/03/11
# a list : str->float
def floatList(l):
    lNew = []
    for s in l:
        lNew.append(float(s))
    #lNew[-1] = int(lNew[-1])
    return lNew

# 2018/03/08
# calculate the distance between two points
def costCal(a,b):
    costAB = 0
    for i in range(dimension):
        costAB = costAB + (a[i]-b[i])**2
    return costAB
    #return math.sqrt(costAB)

# 2018/03/08
# find the cluster which a point should be in
def findCluster(a):
    costMin = 999999
    cluNum  = -1
    for z in range(k):
        costTemp = costCal(a,centers[z])
        if costTemp < costMin:
            costMin = costTemp
            cluNum  = z
    return cluNum

# update a cluster number
def updateCluster(a):
    aNew = []
    for i in range(dimension):
        aNew.append(a[i])
    aNew.append(findCluster(a))
    return aNew

# add up two points
def addXY(x,y):
    z = []
    for i in range(dimension):
        z.append(x[i]+y[i])
    z.append(x[dimension] + 1)
    return z

# sumT is [sum[0],sum[1]...sum[dimension-1],cnt+no-1 ] 
# the result is the center point( len(cPoint) = dimension , with no label) 
def getAverage(sumT,no):
    cnt = sumT[dimension] - no + 1
    cPoint = []
    for i in range(dimension):
        cPoint.append(sumT[i] / cnt)
    return cPoint
 
# cP is the center point of the cluster which both x and y are in
# calculate the distance between y and cP then add x
def addCostXY(xp,yp,cP):
    costS = []
    for i in range(dimension):
        costS.append(xp[i]+(yp[i]-cP[i])**2)
    costS.append(xp[dimension])
    return costS

# costTempList is a list of cost, try to add it up
def addUp(costTempList,headElement,cP):
    sumC = 0
    for i in range(dimension):
        sumC += costTempList[i]
        sumC = sumC + (headElement[i]-cP[i])**2 
        sumC = sumC - headElement[i]
    return sumC

# others......
# 2018/03/10 version 2
# verify: x[dimension] is the cluster number calculated by the programme 
# and result is the origin cluster a point belongs to
def compareNum(x):
    result = findCluster(x)
    return [x[dimension]-1,result]

In [4]:
# k-means++ part
# only for computing the cost list
def costCom(x,s,tempCs):
    cost_Min = 999999
    for c in tempCs:
        cost_Temp = costCal(x,c)
        if cost_Temp < cost_Min:
            cost_Min = cost_Temp
    return (s+cost_Min)

def initialCenters(cCount,allPoints):
    tempCenters = []
    pointCount  = len(allPoints)
    allCost = [0 for i in range(pointCount)]
    
    # pick the first center point
    pickCount   = 1
    tempCenters.append(allPoints[random.randint(0,pointCount-1)])
    
    # pick points until k
    while(pickCount<cCount):
        sumNow  = 0
        allCost = [costCom(allPoints[i],sumNow,tempCenters) for i in range(pointCount)]
        randPick = random.randint(0,allCost[-1])
        for i in range(pointCount):
            if allCost[i] >= randPick:
                tempCenters.append(allPoints[i])
                break
        pickCount += 1
        
    return tempCenters
        

In [5]:
# processing......
timeA = time.time()

In [6]:
# find data 
# get data from files 
data = sc.textFile(path).map(lambda line: line.split(splitChar)) # each line => a list (not a tuple)
data = data.map(lambda x: floatList(x)) # a str list => a int list

# # parallelize
#dataList = createData(1000)
#data = sc.parallelize(dataList)#.map(intList)

# a test......
# data = sc.parallelize(data.takeSample(False, 2000, 1761))

origin = data.map(lambda x: x)

for i in range(k):
    print("Cluster " + str(i) + " : " + str(data.filter(lambda x: x[dimension]==i+1).count()))

Cluster 0 : 50859
Cluster 1 : 194198


In [7]:
# choose centers
centers = [[197.0, 194.0, 163.0], [42.0, 80.0, 62.0]]
#[[53, 59, 47], [157, 165, 157]]

# k-means++
allP = data.map(lambda x:x[0:-1]).collect()
#centers = initialCenters(k,allP)

# randomly
#centers = data.takeSample(False,k,4113)

print("Start with these " + str(k) +" points:")
print(centers)

Start with these 2 points:
[[197.0, 194.0, 163.0], [42.0, 80.0, 62.0]]


In [8]:
# round 1
data = data.map(lambda x: updateCluster(x))
roundCount += 1

costTotal = 0
for i in range(k):
    clusterTempCost = data.filter(lambda x: x[dimension]==i).map(lambda x: costCal(x,centers[i])).sum()
    costTotal += clusterTempCost
#print("First round, cost = " + str(costTotal))

center = copy.deepcopy(centers)

while True:
    if roundCount ==1:
        costTotalTemp = 0
        for i in range(k):
            # find new clusters
            costTemp = data.filter(lambda x: x[dimension]==i).map(lambda x: costCal(x,center[i])).sum()
            costTotalTemp += costTemp
        print("Temp cost: " + str(costTotalTemp) + " version 2")
        
    # calculate new centers
    for i in range(k):
        cluster = data.filter(lambda x: x[dimension]== i)  # origin list
        print("Round " + str(roundCount) + " cluster " + str(i) + " : " + str(cluster.count()))
        # calculate a new center
        sumTempList = cluster.reduce(lambda x,y : addXY(x,y))
        center[i]   = getAverage(sumTempList,i)
        # print("New center:")
        # print(center[i])
    
    # update clustering
    data = data.map(lambda x: updateCluster(x))
    
    # calculate new cost
    # 2018/03/09 version 2
    costTotalTemp = 0
    for i in range(k):
        # find new clusters
        costTemp = data.filter(lambda x: x[dimension]==i).map(lambda x: costCal(x,center[i])).sum()
        costTotalTemp += costTemp
    print("Temp cost: " + str(costTotalTemp) + " version 2")
        
    # converge or not
    if costTotalTemp < (1-threshold)*costTotal:
        centers    = copy.deepcopy(center)
        costTotal  = costTotalTemp
        roundCount += 1
        print("cost: " + str(costTotal))
    else:
        data = data.map(lambda x:updateCluster(x))
        print("Iteration :" + str(roundCount) + " times.")
        break

print(centers)


Temp cost: 1663230078.0 version 2
Round 1 cluster 0 : 152603
Round 1 cluster 1 : 92454
Temp cost: 1380379967.89 version 2
cost: 1380379967.89
Round 2 cluster 0 : 162018
Round 2 cluster 1 : 83039
Temp cost: 1334631853.99 version 2
cost: 1334631853.99
Round 3 cluster 0 : 165848
Round 3 cluster 1 : 79209
Temp cost: 1324887933.38 version 2
cost: 1324887933.38
Round 4 cluster 0 : 167426
Round 4 cluster 1 : 77631
Temp cost: 1322778266.46 version 2
cost: 1322778266.46
Round 5 cluster 0 : 168059
Round 5 cluster 1 : 76998
Temp cost: 1322431232.65 version 2
cost: 1322431232.65
Round 6 cluster 0 : 168342
Round 6 cluster 1 : 76715
Temp cost: 1322370994.77 version 2
Iteration :6 times.
[[157.9789478694982, 165.70890580093896, 157.47164388696828], [53.22708382035897, 60.040195849242835, 48.324579859217124]]


In [9]:
timeB = time.time()

# execution time
print(timeB-timeA)

313.89820981


In [10]:
# verify version 2
# 2018/03/10
aT = [[0 for i in range(k)] for j in range(k)]
compareSet = origin.map(lambda x: compareNum(x)) #x[0]:origin cluster number, x[1]:computed result. both [0,k-1]
print(compareSet.count())
for mm in range(k):
    for nn in range(k):
        aT[mm][nn] = compareSet.filter(lambda x: (x[0]==mm and x[1]==nn)).count()

print(aT)

#accurate = aT[][] + 
#print(accurate)

#accuracy   = 1.0 * accurate / compareSet.count()
#print(accuracy)
print(compareSet.map(lambda x:x[0]).distinct().collect())
print(compareSet.map(lambda x:x[1]).distinct().collect())

245057
[[42016, 8843], [126326, 67872]]
[0.0, 1.0]
[0, 1]


In [11]:
# using  MLlib
from pyspark.ml.clustering import KMeans,KMeansModel
from pyspark.ml.linalg import Vectors
import time

# a list : str->int
def intList(l):
    lNew = []
    for s in l:
        lNew.append(int(s))
    return lNew

kmeans    = KMeans(k=2,seed=1)
newData   = sc.textFile("Skin_NonSkin.txt").map(lambda line: line.split("\t"))
dataL     = newData.map(lambda x:intList(x)[0:-1]).collect()

# into vector
dataV = []
for i in dataL:
    dataV.append((Vectors.dense(i),))

timeA = time.time()
df = spark.createDataFrame(dataV,["features"])
model  = kmeans.fit(df)
centersNew = model.clusterCenters()
timeB = time.time()
print (centersNew)
print(timeB-timeA)

[array([53.10845893, 59.80089936, 47.65160329]), array([157.72267706, 165.50468681, 157.45395057])]
34.8363540173
